### My project on dataquest 

For this project, we'll pretend we're working as **data analysts** for a company that builds Android and iOS mobile apps.
We make our apps available on Google Play and the App Store.

We only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means our revenue for any given app is mostly influenced by the number of users who use our app — **the more users that see and engage with the ads, the better.** Our goal for this project is to analyze data *to help our developers understand what type of apps are likely to attract more users.*

In [1]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [2]:
opened_file1 = open('AppleStore.csv', encoding='utf-8')
from csv import reader
read_file1 = reader(opened_file1)
apps_data = list(read_file1)

opened_file2 = open('googleplaystore.csv', encoding='utf-8')
from csv import reader
read_file2 = reader(opened_file2)
google_data = list(read_file2)

In [3]:
explore_data(apps_data, 0, 3, rows_and_columns=True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7198
Number of columns: 16


In [4]:
explore_data(google_data, 0, 3, rows_and_columns=True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


In [5]:
# удаляем ряд, в котором обнаружена ошибка (отсутствует столбец category)
del google_data[10473]

In [6]:
print(google_data[10473])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


**As our datasets have duplicates we count them and delete the oldest of them**

In [7]:
duplicate_google = []
unique_google  = []

for row in google_data[1:]:
    name = row[0]
    if name in unique_google:
        duplicate_google.append(name)
    else:
        unique_google.append(name)
        
print('Number of duplicate google apps:', len(duplicate_google))
print('\n')
print('Examples of duplicate google apps:', duplicate_google[:5])

Number of duplicate google apps: 1181


Examples of duplicate google apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


In [8]:
duplicate_apps = []
unique_apps  = []

for row in apps_data[1:]:
    name = row[1]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apple apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apple apps:', duplicate_apps[:5])

Number of duplicate apple apps: 2


Examples of duplicate apple apps: ['Mannequin Challenge', 'VR Roller Coaster']


**it seems that our apps_data has only 2 duplicates**

**For our google_data**
*The higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given app.*

In [9]:
reviews_max = {}
for row in google_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews
len(reviews_max)

9659

In [10]:
google_clean = []
already_added = []
for row in google_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        google_clean.append(row)
        already_added.append(name)
len(google_clean)

9659

In [11]:
ratings_max = {}
for row in apps_data[1:]:
    name = row[1]
    n_ratings = float(row[5])
    if name in ratings_max and ratings_max[name] < n_ratings:
        ratings_max[name] = n_ratings
    if name not in ratings_max:
        ratings_max[name] = n_ratings
len(ratings_max)

7195

In [12]:
apple_clean = []
already_added = []
for row in apps_data[1:]:
    name = row[1]
    n_ratings = float(row[5])
    if n_ratings == ratings_max[name] and name not in already_added:
        apple_clean.append(row)
        already_added.append(name)
len(apple_clean)

7195

In the previous step, we managed to remove the duplicate app entries in the Google Play data set. Remember we use English for the apps we develop at our company, and we'd like to analyze only the apps that are directed toward an English-speaking audience. However, if we explore the data long enough, we'll find that both data sets have apps with names that suggest they are not directed toward an English-speaking audience.

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

In [13]:
# function takes in a string and returns False
# if there's any character in the string that doesn't belong
# to the set of common English characters, otherwise it returns True

def checking_english(string):
    for letter in string:
        if ord(letter) > 127:
            return False
    return True

In [14]:
print(checking_english('Instagram'))
print(checking_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(checking_english('Docs To Go™ Free Office Suite'))
print(checking_english('Instachat 😜'))

True
False
False
False


If we're going to use the function we've created, we'll lose useful data since many English apps will be incorrectly labeled as non-English. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [15]:
# updating our function

def checking_english_v2(string):
    count_letters = 0
    for letter in string:
        if ord(letter) > 127:
            count_letters += 1
    if count_letters > 3:
        return False
    else:
        return True

In [16]:
print(checking_english_v2('Instagram'))
print(checking_english_v2('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(checking_english_v2('Docs To Go™ Free Office Suite'))
print(checking_english_v2('Instachat 😜'))

True
False
True
True


*Using the new function to filter out non-English apps from both data sets*

In [17]:
google_clean_eng = []
for row in google_clean:
    name = row[0]
    if checking_english_v2(name) == True:
        google_clean_eng.append(row)
len(google_clean_eng)

9614

In [18]:
apple_clean_eng = []
for row in apple_clean:
    name = row[1]
    if checking_english_v2(name) == True:
        apple_clean_eng.append(row)
len(apple_clean_eng)

6181

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps; we'll need to isolate only the free apps for our analysis.

In [19]:
free_google_clean_eng = []
paid_google_clean_eng = []
for row in google_clean_eng:
    type_price = row[6]
    if type_price == 'Free':
        free_google_clean_eng.append(row)
    else:
        paid_google_clean_eng.append(row)
print(len(free_google_clean_eng))
print(len(paid_google_clean_eng))

8863
751


In [20]:
free_apple_clean_eng = []
paid_apple_clean_eng = []
for row in apple_clean_eng:
    price = float(row[4])
    if price == 0:
        free_apple_clean_eng.append(row)
    else:
        paid_apple_clean_eng.append(row)
print(len(free_apple_clean_eng))
print(len(paid_apple_clean_eng))

3220
2961


Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

In [21]:
# формируем словарь с количеством жанров
google_genres = {}
for row in free_google_clean_eng:
    genre = row[1]
    if genre in google_genres:
        google_genres[genre] += 1
    else:
        google_genres[genre] = 1

# выполняем сортировку словаря по значениям и делаем реверс, печатаем топ-10
list_google_genres = list(google_genres.items())
list_google_genres.sort(key=lambda i: i[1])
list_google_genres.reverse()
for i in list_google_genres[:10]:
    print(i[0], ':', i[1])

FAMILY : 1675
GAME : 862
TOOLS : 750
BUSINESS : 407
LIFESTYLE : 346
PRODUCTIVITY : 345
FINANCE : 328
MEDICAL : 313
SPORTS : 301
PERSONALIZATION : 294


In [22]:
# формируем словарь с количеством жанров
apple_genres = {}
for row in free_apple_clean_eng:
    genre = row[11]
    if genre in apple_genres:
        apple_genres[genre] += 1
    else:
        apple_genres[genre] = 1

# выполняем сортировку словаря по значениям и делаем реверс, печатаем топ-10
list_apple_genres = list(apple_genres.items())
list_apple_genres.sort(key=lambda i: i[1])
list_apple_genres.reverse()
for i in list_apple_genres[:10]:
    print(i[0], ':', i[1])

Games : 1872
Entertainment : 254
Photo & Video : 160
Education : 118
Social Networking : 106
Shopping : 84
Utilities : 81
Sports : 69
Music : 66
Health & Fitness : 65


The frequency tables we analyzed on the previous screen showed us that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and fun apps. Now, we'd like to get an idea about the kind of apps with the most users.

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

In [23]:
# считаем среднее количество оценок пользователей для каждого жанра
for genre in apple_genres:
    total = 0
    len_genre = 0
    
    for row in free_apple_clean_eng:
        genre_app = row[11]
        if genre_app == genre:
            n_rating = float(row[5])
            total += n_rating
            len_genre += 1
    
    avg_number = round(total / len_genre, 1)
    print(genre, avg_number)


Social Networking 71548.3
Photo & Video 28441.5
Games 22812.9
Music 57326.5
Reference 74942.1
Health & Fitness 23298.0
Weather 52279.9
Utilities 18684.5
Travel 28243.8
Shopping 26919.7
News 21248.0
Navigation 86090.3
Lifestyle 16485.8
Entertainment 14029.8
Food & Drink 33333.9
Sports 23008.9
Book 39758.5
Finance 31467.9
Education 7004.0
Productivity 21028.4
Business 7491.1
Catalogs 4004.0
Medical 612.0


In [24]:
# считаем среднее количество установок для каждого жанра
for genre in google_genres:
    total = 0
    len_genre = 0
    
    for row in free_google_clean_eng:
        genre_app = row[1]
        if genre_app == genre:
            n_installs = row[5]
            n_installs = n_installs.replace('+', '')
            n_installs = n_installs.replace(',', '')
            n_installs = float(n_installs)
            total += n_installs
            len_genre += 1
    
    avg_number = round(total / len_genre, 1)
    print(genre, avg_number)


ART_AND_DESIGN 1986335.1
AUTO_AND_VEHICLES 647317.8
BEAUTY 513151.9
BOOKS_AND_REFERENCE 8767811.9
BUSINESS 1712290.1
COMICS 817657.3
COMMUNICATION 38456119.2
DATING 854028.8
EDUCATION 1833495.1
ENTERTAINMENT 11640705.9
EVENTS 253542.2
FINANCE 1387692.5
FOOD_AND_DRINK 1924897.7
HEALTH_AND_FITNESS 4188822.0
HOUSE_AND_HOME 1331540.6
LIBRARIES_AND_DEMO 638503.7
LIFESTYLE 1437816.3
GAME 15588015.6
FAMILY 3697848.2
MEDICAL 120550.6
SOCIAL 23253652.1
SHOPPING 7036877.3
PHOTOGRAPHY 17840110.4
SPORTS 3638640.1
TRAVEL_AND_LOCAL 13984077.7
TOOLS 10801391.3
PERSONALIZATION 5201482.6
PRODUCTIVITY 16787331.3
PARENTING 542603.6
WEATHER 5074486.2
VIDEO_PLAYERS 24727872.5
NEWS_AND_MAGAZINES 9549178.5
MAPS_AND_NAVIGATION 4056941.8
